**6.- Entrena un modelo en Keras que combine capas GRU y convolucionales para predecir los valores de cierre de Apple utilizando como entrada únicamente los valores de apertura de los datos Apple de los 4 días anteriores y que obtenga unos resultados en test mejores que los del apartado anterior.**


In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Layer, Dense, GRU, Conv1D, Flatten, Dropout, BatchNormalization, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
#Leemos los datos de apple
apple = pd.read_csv(r"C:\Users\Carlos\Documents\MIA-X\Modulo 4\Machine Learning\Redes Convulcionales\CNN + RNN\practica_rnn_cnn\Stocks\aa.us.txt", delimiter=",")
#Me quedo con los datos de apertura
open_data = apple['Open'].values
close_data = apple['Close'].values

In [3]:
#Me genero una matriz de ceros con las dimensiones que quiero, n_ejemplos x 5 datos. 
LAG = 5
data = np.zeros((open_data.shape[0]- LAG, LAG))
close = np.zeros((close_data.shape[0]- LAG, LAG))

In [4]:
#Inserto en la matriz de ceros los datos que necesitamos con el LAG de 5
for i in range(0,(open_data.shape[0] - LAG)):
    data[i,:] = open_data[i:i+LAG]
    close[i,:] = close_data[i:i+LAG]

In [5]:
data.shape[0]

12069

In [6]:
#Dividimos los datos en x_train, y_train, x_test, y_test, la columna 5 será nuestro label. 
n_data = data.shape[0]
x_train = data[:round(n_data*0.7), :-1]
y_train = data[:round(n_data*0.7), -1]
x_test = close[round(n_data*0.7):, :-1]
y_test = close[round(n_data*0.7):, -1]

In [7]:
#Definimos nuestro modelo
#Usaremos return_sequences true para que las dimensiones encajen con las capas convulcionales
model = Sequential()

model.add(GRU(20, activation = 'relu', input_shape = (4,1), return_sequences = True))
model.add(Conv1D(10, kernel_size = 3, activation = 'relu',  
                 kernel_regularizer=regularizers.l2(0.01),padding='same'))

model.add(Flatten())
model.add(Dense(1, activation = 'linear'))

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 4, 20)             1380      
                                                                 
 conv1d (Conv1D)             (None, 4, 10)             610       
                                                                 
 flatten (Flatten)           (None, 40)                0         
                                                                 
 dense (Dense)               (None, 1)                 41        
                                                                 
Total params: 2,031
Trainable params: 2,031
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',loss='mae')

In [10]:
history_convulcional = model.fit(x_train, y_train,  epochs=50)

Epoch 1/50
264/264 [==============================] - 1s 1ms/step - loss: 3.2626
Epoch 2/50
264/264 [==============================] - 0s 1ms/step - loss: 0.4861
Epoch 3/50
264/264 [==============================] - 0s 1ms/step - loss: 0.4787
Epoch 4/50
264/264 [==============================] - 0s 1ms/step - loss: 0.4583
Epoch 5/50
264/264 [==============================] - 0s 1ms/step - loss: 0.4378
Epoch 6/50
264/264 [==============================] - 0s 1ms/step - loss: 0.4372
Epoch 7/50
264/264 [==============================] - 0s 1ms/step - loss: 0.4187
Epoch 8/50
264/264 [==============================] - 0s 1ms/step - loss: 0.4230
Epoch 9/50
264/264 [==============================] - 0s 1ms/step - loss: 0.4097
Epoch 10/50
264/264 [==============================] - 0s 1ms/step - loss: 0.4059
Epoch 11/50
264/264 [==============================] - 0s 1ms/step - loss: 0.3963
Epoch 12/50
264/264 [==============================] - 0s 1ms/step - loss: 0.3908
Epoch 13/50
264/264 [====

In [11]:
model.evaluate(x_test, y_test)

114/114 [==============================] - 0s 778us/step - loss: 0.7155


0.7154545187950134

Hemos obtenido un resultado similar al del ejercicio anterior.